In [15]:
import subprocess
import io
import os
import numpy as np
import time


In [16]:
# read the pass code from the file
passcode = ""

with open("passcode.txt", "r") as f:
    passcode = f.read()
    f.close()


In [17]:
# unmout the tmpfs
while not os.system('echo '+passcode+' | sudo -S umount ./myTmpfs'):
    pass

[sudo] password for txin: 

[sudo] password for txin: umount: ./myTmpfs: not mounted.


In [18]:
# Write a file in the tmpfs directory
result = subprocess.run(['mkdir', './myTmpfs'], stdout=subprocess.PIPE)
os.system('echo '+passcode +'| sudo -S mount -t tmpfs -o size=10G tmpfs ./myTmpfs')

# Test the io speed of the tmpfs
result = subprocess.run(['dd', 'if=/dev/zero', 'of=./myTmpfs/testin', 'bs=1M', 'count=4096'], stdout=subprocess.PIPE)
print(result.stdout)

mkdir: cannot create directory ‘./myTmpfs’: File exists
[sudo] password for txin: 

b''


4096+0 records in
4096+0 records out
4294967296 bytes (4.3 GB, 4.0 GiB) copied, 1.06904 s, 4.0 GB/s


In [19]:
# Run your subprocess command
#result = subprocess.run(['mkdir', './myTmpfs'], stdout=subprocess.PIPE)
#os.system('echo '+passcode +'| sudo -S mount -t tmpfs -o size=10G tmpfs ./myTmpfs')

# Test the io speed of the tmpfs
result = subprocess.run(['dd', 'if=./myTmpfs/testin', 'of=./myTmpfs/testout', 'bs=1M', 'count=4096'], stdout=subprocess.PIPE)
print(result.stdout)

# Test the io speed of real hard drive
result = subprocess.run(['dd', 'if=./myTmpfs/testin', 'of=./testout', 'bs=1M', 'count=4096'], stdout=subprocess.PIPE)
print(result.stdout)

# unmout the tmpfs
# os.system('echo '+passcode+' | sudo -S umount ./myTmpfs')


4096+0 records in
4096+0 records out
4294967296 bytes (4.3 GB, 4.0 GiB) copied, 1.64616 s, 2.6 GB/s


b''
b''


4096+0 records in
4096+0 records out
4294967296 bytes (4.3 GB, 4.0 GiB) copied, 7.80268 s, 550 MB/s


In [20]:
def write_test(file_path, data_size):
    # write a binary file with 'data_size' bytes
    data = os.urandom(data_size)
    print(f'Writing {data_size} bytes to {file_path}')
    start_time = time.time()
    with open(file_path, 'wb') as file:
        # write the binaary file
        file.write(data)
        file.close()
    end_time = time.time()
    return end_time - start_time

def read_test(file_path):
    start_time = time.time()
    with open(file_path, 'rb') as file:
        data = file.read()
        file.close()
    end_time = time.time()
    return end_time - start_time

# Path to the file in tmpfs

file_path_tmpfs = './myTmpfs/test_file'

file_path_ssd = './test_file'
#file_path_memory = './myTmpfs/test_file'
data_size = 100 * 1024 * 1024

# Perform the write test
write_time = write_test(file_path_tmpfs, data_size)
print(f'Write time for {data_size} bytes: {write_time} seconds')
print(f'Write speed for {data_size} bytes: {data_size / write_time / 1024 / 1024 / 1024} GB/s')
# Perform the read test
read_time = read_test(file_path_tmpfs)
print(f'Read time for {data_size} bytes: {read_time} seconds')
print(f'Read speed for {data_size} bytes: {data_size / read_time / 1024 / 1024 / 1024} GB/s')
# Optionally, clean up by removing the test file
# os.remove(file_path_tmpfs)

# Perform the write test on SSD
write_time = write_test(file_path_ssd, data_size)
print(f'Write time for {data_size} bytes: {write_time} seconds')
print(f'Write speed for {data_size} bytes: {data_size / write_time / 1024 / 1024 / 1024} GB/s')
# Perform the read test
read_time = read_test(file_path_ssd)
print(f'Read time for {data_size} bytes: {read_time} seconds')
print(f'Read speed for {data_size} bytes: {data_size / read_time / 1024 / 1024 / 1024} GB/s')



Writing 104857600 bytes to ./myTmpfs/test_file
Write time for 104857600 bytes: 0.04070234298706055 seconds
Write speed for 104857600 bytes: 2.399278342061177 GB/s
Read time for 104857600 bytes: 0.10785841941833496 seconds
Read speed for 104857600 bytes: 0.9054114692821034 GB/s
Writing 104857600 bytes to ./test_file
Write time for 104857600 bytes: 0.15537524223327637 seconds
Write speed for 104857600 bytes: 0.6285187305026462 GB/s
Read time for 104857600 bytes: 0.09858059883117676 seconds
Read speed for 104857600 bytes: 0.9906234204079066 GB/s


In [21]:
def memory_write_test(size):
    # Create a large array with random data
    data = np.random.rand(size, size)
    start_time = time.time()
    # Perform a write operation (e.g., copying the array)
    new_data = np.copy(data)
    end_time = time.time()
    return end_time - start_time

def memory_read_test(array):
    start_time = time.time()
    # Perform a read operation (e.g., summing the array)
    total = np.sum(array)
    end_time = time.time()
    return end_time - start_time

# Define the size of the array (e.g., 1000x1000 elements)
array_size = 30000

# Test memory write speed
write_time = memory_write_test(array_size)
print(f'Memory write test for {array_size}x{array_size} array: {write_time} seconds')
print(f'Memory write speed for {array_size}x{array_size} array: {array_size * array_size / write_time / 1024 / 1024 / 1024} GB/s')
# Create an array for the read test
test_array = np.random.rand(array_size, array_size)

# Test memory read speed
read_time = memory_read_test(test_array)
print(f'Memory read test for {array_size}x{array_size} array: {read_time} seconds')
print(f'Memory read speed for {array_size}x{array_size} array: {array_size * array_size / read_time / 1024 / 1024 / 1024} GB/s')


Memory write test for 30000x30000 array: 6.274712085723877 seconds
Memory write speed for 30000x30000 array: 0.13358227528255323 GB/s
Memory read test for 30000x30000 array: 0.42903923988342285 seconds
Memory read speed for 30000x30000 array: 1.9536448866114928 GB/s


In [ ]:
# read the json file
